In [1]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from typing import Union
from dataclasses import dataclass

from copy import deepcopy
import json

from case_edits.ezcase import EzCase

from recipes.amber import input
from recipes.exp_01 import input as input1, ns_axis
from geometry.wall_normal import WallNormal
from helpers.helpers import key_from_value
from methods.subsurfaces.pairs import DEFAULT_WINDOW, SubsurfacePair, DEFAULT_DOOR


from gplan.subsurface_translator import SubsurfaceTranslator

In [2]:
# input_ns = ns_axis(input1)
# input_ns.subsurface_pairs

In [2]:
ez = EzCase(input, RUN_CASE=False)
ez

tests/test21_amber


IndexError: list index out of range

In [7]:
ineligible_zones = [k for k,v in ez.afn.cnt.items() if v < 2]
for s  in ez.afn.subsurface_zones:
    if s.zone in ineligible_zones or s.partner_zone in ineligible_zones:
        print(s.subsurface.name)



Block 03 Storey 0 Wall 0004_1 Door
Block 05 Storey 0 Wall 0003_1 Door


In [9]:
ez.afn.subsurfaces[0].Name

'Block 00 Storey 0 Wall 0001_1 Door'

In [12]:
'Block 05 Storey 0' in [z.Zone_Name for z in ez.case.idf.idfobjects["AIRFLOWNETWORK:MULTIZONE:ZONE"]]

False

In [5]:
ez.show_base_plot()

In [2]:
st = SubsurfaceTranslator("amber_a")
st.run()

In [13]:
b1 = "B_010"
nums= b1.split("_")[1]

nums[-2:]

'10'

In [14]:
def convert_block_name_to_int(name):
    nums = name.split("_")[1]
    match len(nums):
        case 2:
            return int(nums[-1])
        case 3:
            return int(nums[-2:])
        case _:
            raise Exception("invalid name length")

In [21]:
convert_block_name_to_int("B_0101")

Exception: invalid name length

In [6]:
st.door_pairs + st.window_pairs

[SubsurfacePair(pair=(0, 7), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(1, 6), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(2, 3), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(3, 4), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADI

In [29]:
def create_numeric_name(block):
    label = block["label"]
    id  = block["id"]
    try:
        label = int(label) + 0
        name = f"0{label}"
    except ValueError:
        name = f"0{id}"

    return name

In [30]:
block = {"label": 8, "id": 0}

In [31]:
create_numeric_name(block)

'08'

In [2]:
with open(PLAN_PATH) as f:
    plan_data = json.load(f)

In [3]:
room_map = {}
for item in plan_data[0]:
    room_map[item["id"]] = item["label"]

room_map

{0: 'bedroom_1',
 1: 'kitchen',
 2: 'm_bath',
 3: 'm_bedroom',
 4: 'm_closet',
 5: 'bath',
 6: 'dining',
 7: 'transit_space',
 8: 'closet',
 9: 'bedroom_2',
 10: 'laundry'}

In [4]:
CONNECTIVITY_PATH = "/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/outputs/amber_a/graph.json"

In [5]:
with open(CONNECTIVITY_PATH) as f:
    data = json.load(f)

In [6]:
@dataclass
class InputObject:
    node: Union[str, int]
    direction: str
    object_type: str

In [7]:
objects: list[InputObject] = []
for node in data["nodes"]:
    for w in node["windows"]:
        objects.append(InputObject(node["id"],
                        w["direction"],
                         w["window_type"]
        ))

In [8]:
WallNormal["NORTH"]

<WallNormal.NORTH: 0.0>

In [9]:
window_pairs: list[SubsurfacePair] = []
for o in objects:
    node = key_from_value(room_map, o.node) 
    direction = WallNormal[str(o.direction)]
    if o.object_type == "A":
        object_type = DEFAULT_WINDOW

    window_pairs.append(SubsurfacePair(node, direction, object_type))

In [11]:
doors = data["links"]
doors

[{'source': 'bedroom_1', 'target': 'transit_space'},
 {'source': 'kitchen', 'target': 'dining'},
 {'source': 'm_bath', 'target': 'm_bedroom'},
 {'source': 'm_bedroom', 'target': 'm_closet'},
 {'source': 'm_bedroom', 'target': 'dining'},
 {'source': 'bath', 'target': 'transit_space'},
 {'source': 'dining', 'target': 'transit_space'},
 {'source': 'dining', 'target': 'closet'},
 {'source': 'transit_space', 'target': 'bedroom_2'},
 {'source': 'transit_space', 'target': 'laundry'}]

In [14]:
door_pairs: list[SubsurfacePair] = []
for item in doors:
    door_pairs.append(SubsurfacePair(
        key_from_value(room_map, item["source"]),
        key_from_value(room_map, item["target"]),
        DEFAULT_DOOR
    )
    )

In [15]:
door_pairs

[SubsurfacePair(pair=(0, 7), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(1, 6), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(2, 3), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(3, 4), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADI

tests/test21_amber
Added 5 walls 
Added 4 walls 
Added 4 walls 
Added 6 walls 
Added 4 walls 
Added 5 walls 
Added 7 walls 
Added 6 walls 
Added 4 walls 
Added 5 walls 
Added 5 walls 


AttributeError: 'list' object has no attribute 'attrs'

In [5]:
ez.show_base_plot()